# Energy Evaluation

### Define Energy Intesities

In [74]:
TRACE_RETENTION_HOURS = 360 # 30 days

# energy intensity kWh/GB
ENERGY_INTENSITY_NETWORK_KWH_PER_GB = 0.001875
ENERGY_INTENSITY_STORAGE_WH_PER_TB_HOUR = 0.525 * TRACE_RETENTION_HOURS

## Load pre-processed data
This data can be pre-processed with the help of kepler-evaluation.ipynb (CPU+RAM), network-evaluation.ipynb and storage-evaluation.ipynb. If you want to do new experiments, you have to create these files yourself in the meantioned jupyter notebooks.

In [75]:
import pandas as pd
import tabulate as tabulate
network_data = pd.read_csv("cadvisor_network_bytes_received_all_absolute_bytes.csv", index_col=0)
network_data_transmitted = pd.read_csv("cadvisor_network_bytes_transmitted_all_absolute_bytes.csv", index_col=0)
storage_data = pd.read_csv("cadvisor_storage_usage_writes_all_absolute_bytes.csv", index_col=0)
kepler_data_joules = pd.read_csv("pods_kepler_joules_all_absolute_joules.csv", index_col=0)
kepler_dram = pd.read_csv("pods_kepler_dram_joules_all_joules.csv", index_col=0)


## Network
Further process the network data to transform total bytes received to kWh using the energy intensity

In [76]:
# get the last value for each label based on the time_normalized_rounded as this is the total amount of data received
network_data_processed = network_data.groupby("label").last().reset_index()


# remove the column time_normalized_rounded as it is not needed anymore
network_data_processed = network_data_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
network_data_processed.columns = network_data_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

network_data_processed = network_data_processed.set_index("label")

network_data_processed = network_data_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
    "Base Persistence",
    "Scrape 5s Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "Persistence Istio",
])

# drop indexes the experiments that are not needed e.g. Baseline, 0%, 5%, 10%, 50%, 1s, 30s
network_data_processed = network_data_processed.drop(index=[
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
])

# rename the columns to be more readable
network_data_processed = network_data_processed.rename(index={
    "Base Persistence": "Baseline",
    "Scrape 5s Persistence": "Scrape 5s",
    "5% Persistence": "5%",
    "10% Persistence": "10%",
    "50% Persistence": "50%",
    "Persistence Istio": "Istio",
})



# this values are in bytes, convert them to GB
network_data_processed = network_data_processed / (1000 * 1000 * 1000)


print("Total network data received per Experiment (label) in GB:")
print(network_data_processed.round(2).to_markdown())


# calculate the energy consumption in kWh
network_data_processed = network_data_processed * ENERGY_INTENSITY_NETWORK_KWH_PER_GB


# print as table
print("Total network data received per Experiment (label) and energy consumption in kWh:")
print(network_data_processed.to_markdown())
print("Total energy consumption for storage in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in network_data_processed.index:
    if label == "Baseline":
        print(f"{label}: {network_data_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (network_data_processed.loc[label].sum() / network_data_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {network_data_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")

Total network data received per Experiment (label) in GB:
| label     |   accounting |   ad |   cart |   checkout |   currency |   elasticsearch |   email |   flagd |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   jaeger |   jaeger-collector |   jaeger-query |   kafka |   otel-collector |   payment |   product-catalog |   prometheus |   quote |   recommendation |   shipping |   valkey-cart |
|:----------|-------------:|-----:|-------:|-----------:|-----------:|----------------:|--------:|--------:|------------------:|-----------:|-----------------:|----------:|-----------------:|---------:|-------------------:|---------------:|--------:|-----------------:|----------:|------------------:|-------------:|--------:|-----------------:|-----------:|--------------:|
| Baseline  |            0 | 0.01 |   0.05 |          0 |          0 |            0.18 |       0 |    0.02 |                 0 |       0.71 |             0.89 |         0 |                0 | 

In [77]:
# get the last value for each label based on the time_normalized_rounded as this is the total amount of data received
network_data_transmitted_processed = network_data_transmitted.groupby("label").last().reset_index()


# remove the column time_normalized_rounded as it is not needed anymore
network_data_transmitted_processed = network_data_transmitted_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
network_data_transmitted_processed.columns = network_data_transmitted_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

network_data_transmitted_processed = network_data_transmitted_processed.set_index("label")

network_data_transmitted_processed = network_data_transmitted_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
    "Base Persistence",
    "Scrape 5s Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "Persistence Istio"
])

# drop indexes the experiments that are not needed e.g. Baseline, 0%, 5%, 10%, 50%, 1s, 30s
network_data_transmitted_processed = network_data_transmitted_processed.drop(index=[
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
])

# rename the columns to be more readable
network_data_transmitted_processed = network_data_transmitted_processed.rename(index={
    "Base Persistence": "Baseline",
    "Scrape 5s Persistence": "Scrape 5s",
    "5% Persistence": "5%",
    "10% Persistence": "10%",
    "50% Persistence": "50%",
    "Persistence Istio": "Istio",
})


# this values are in bytes, convert them to GB
network_data_transmitted_processed = network_data_transmitted_processed / (1000 * 1000 * 1000)


print("Total network data transmitted per Experiment (label) in GB:")
print(network_data_transmitted_processed.round(2).to_markdown())


# calculate the energy consumption in kWh
network_data_transmitted_processed = network_data_transmitted_processed * ENERGY_INTENSITY_NETWORK_KWH_PER_GB


# print as table
print("Total network data transmitted per Experiment (label) and energy consumption in kWh:")
print(network_data_transmitted_processed.to_markdown())
print("Total energy consumption for storage in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in network_data_transmitted_processed.index:
    if label == "Baseline":
        print(f"{label}: {network_data_transmitted_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (network_data_transmitted_processed.loc[label].sum() / network_data_transmitted_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {network_data_transmitted_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")

Total network data transmitted per Experiment (label) in GB:
| label     |   accounting |   ad |   cart |   checkout |   currency |   elasticsearch |   email |   flagd |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   jaeger |   jaeger-collector |   jaeger-query |   kafka |   otel-collector |   payment |   product-catalog |   prometheus |   quote |   recommendation |   shipping |   valkey-cart |
|:----------|-------------:|-----:|-------:|-----------:|-----------:|----------------:|--------:|--------:|------------------:|-----------:|-----------------:|----------:|-----------------:|---------:|-------------------:|---------------:|--------:|-----------------:|----------:|------------------:|-------------:|--------:|-----------------:|-----------:|--------------:|
| Baseline  |            0 | 0.13 |   0.33 |          0 |          0 |            0    |       0 |    0.05 |                 0 |       1.79 |             2.04 |         0 |                0

## Storage
Further process the storage data to transform total bytes used to kWh using the energy intensity

In [78]:
storage_data_processed = storage_data.groupby("label").last().reset_index()

# remove the column time_normalized_rounded as it is not needed anymore
storage_data_processed = storage_data_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
storage_data_processed.columns = storage_data_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

storage_data_processed = storage_data_processed.set_index("label")

# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
storage_data_processed = storage_data_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
    "Base Persistence",
    "Scrape 5s Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "Persistence Istio",
])

# drop indexes the experiments that are not needed e.g. Baseline, 0%, 5%, 10%, 50%, 1s, 30s
storage_data_processed = storage_data_processed.drop(index=[
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
])

# rename the columns to be more readable
storage_data_processed = storage_data_processed.rename(index={
    "Base Persistence": "Baseline",
    "Scrape 5s Persistence": "Scrape 5s",
    "5% Persistence": "5%",
    "10% Persistence": "10%",
    "50% Persistence": "50%",
    "Persistence Istio": "Istio",
})

# rename opentelemetry-collector to otel-collector
storage_data_processed = storage_data_processed.rename(index={"opentelemetry-collector": "otel-collector"})


# this values are in bytes, convert them to TB
storage_data_processed = storage_data_processed / (1000 * 1000 * 1000 * 1000)



print("Storage data used per Experiment (label) in GB:")
print(storage_data_processed.multiply(1000).to_markdown())

# calculate the energy consumption in kWh
storage_data_processed = (storage_data_processed * ENERGY_INTENSITY_STORAGE_WH_PER_TB_HOUR) / 1000




# print as table
print("Total energy consumption for storage in kWh per Experiment (label):")
print(storage_data_processed.to_markdown())
print("Total energy consumption for storage in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in storage_data_processed.index:
    if label == "Baseline":
        print(f"{label}: {storage_data_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (storage_data_processed.loc[label].sum() / storage_data_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {storage_data_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")


Storage data used per Experiment (label) in GB:
| label     |   accounting |         ad |   elasticsearch |   email |   flagd |   fraud-detection |   frontend |   frontend-proxy |     grafana |   image-provider |     kafka |   opensearch |   opentelemetry-collector |   payment |   prometheus |   recommendation |   valkey-cart |
|:----------|-------------:|-----------:|----------------:|--------:|--------:|------------------:|-----------:|-----------------:|------------:|-----------------:|----------:|-------------:|--------------------------:|----------:|-------------:|-----------------:|--------------:|
| Baseline  |            0 | 0.00730726 |         1.21453 |       0 |       0 |        0.00667648 |          0 |                0 | 0.000303104 |                0 | 0.0331899 |      5.67384 |                       nan |         0 |    0.0306053 |                0 |    9.8304e-05 |
| Scrape 5s |            0 | 0.00734003 |         1.17649 |       0 |       0 |        0.00670106 |       

Further process the CPU energy data. We use the average over the experiment as the kWh

In [79]:
""" kepler_data_processed = kepler_data.groupby("label").mean()

# remove the column time_normalized_rounded as it is not needed anymore
kepler_data_processed = kepler_data_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
kepler_data_processed.columns = kepler_data_processed.columns.str.replace("_Auxiliary-under-evaluation", "", regex=False)

# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
kepler_data_processed = kepler_data_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "Base Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "Base Persistence Istio"
])

# print as table
print("Total energy consumption in kWh per Experiment (label):")
print(kepler_data_processed.to_markdown())
print("Total energy consumption in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in kepler_data_processed.index:
    if label == "Baseline":
        print(f"{label}: {kepler_data_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (kepler_data_processed.loc[label].sum() / kepler_data_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {kepler_data_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)") """

' kepler_data_processed = kepler_data.groupby("label").mean()\n\n# remove the column time_normalized_rounded as it is not needed anymore\nkepler_data_processed = kepler_data_processed.drop(columns=["time_normalized_rounded"])\n\n# remove _system-under-evaluation from the column names\nkepler_data_processed.columns = kepler_data_processed.columns.str.replace("_Auxiliary-under-evaluation", "", regex=False)\n\n# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence\nkepler_data_processed = kepler_data_processed.reindex([\n    "Baseline",\n    "0%",\n    "5%",\n    "10%",\n    "50%",\n    "1s",\n    "Base Persistence",\n    "5% Persistence",\n    "10% Persistence",\n    "50% Persistence",\n    "Base Persistence Istio"\n])\n\n# print as table\nprint("Total energy consumption in kWh per Experiment (label):")\nprint(kepler_data_processed.to_markdown())\nprint("Total energy consumption in kWh per Experiment 

In [80]:
kepler_data_joules_processed = kepler_data_joules.groupby("label").last()

# remove the column time_normalized_rounded as it is not needed anymore
kepler_data_joules_processed = kepler_data_joules_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
kepler_data_joules_processed.columns = kepler_data_joules_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

# drop the columns that are not needed
kepler_data_joules_processed = kepler_data_joules_processed.drop(columns=["master", "worker"])


# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
kepler_data_joules_processed = kepler_data_joules_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
    "Base Persistence",
    "Scrape 5s Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "Persistence Istio",
])  

# drop indexes the experiments that are not needed e.g. Baseline, 0%, 5%, 10%, 50%, 1s, 30s
kepler_data_joules_processed = kepler_data_joules_processed.drop(index=[
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
])

# rename the columns to be more readable
kepler_data_joules_processed = kepler_data_joules_processed.rename(index={
    "Base Persistence": "Baseline",
    "Scrape 5s Persistence": "Scrape 5s",
    "5% Persistence": "5%",
    "10% Persistence": "10%",
    "50% Persistence": "50%",
    "Persistence Istio": "Istio",
})

kepler_data_joules_processed_total = kepler_data_joules_processed.copy()
kepler_data_joules_processed_total["Total"] = kepler_data_joules_processed.sum(axis=1)

# print as table
print("Total energy consumption in Joules per Experiment (label):")
print(kepler_data_joules_processed_total.to_markdown())

# convert Joules to kWh
kepler_data_joules_processed = kepler_data_joules_processed / 3600000  # 1 kWh = 3.6 million Joules
print("Total energy consumption in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in kepler_data_joules_processed.index:
    if label == "Baseline":
        print(f"{label}: {kepler_data_joules_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (kepler_data_joules_processed.loc[label].sum() / kepler_data_joules_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {kepler_data_joules_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")


Total energy consumption in Joules per Experiment (label):
| label     |   accounting |      ad |    cart |   checkout |   configfile |   currency |   elasticsearch |   email |   exporter |   flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |   istio-init |   istio-proxy |   jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |   kafka |   opensearch |   opentelemetry-collector |   payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   Total |
|:----------|-------------:|--------:|--------:|-----------:|-------------:|-----------:|----------------:|--------:|-----------:|--------:|-----------:|------------------:|-----------:|-----------------:|----------:|-----------------:|--------------:|-------------:|--------------:|---------:|-----------------------:|-------------------:|---------------:|-------

# DRAM
Does not work as expected currently. Values are strange and mostly negativ.


In [81]:
kepler_ram_joules_processed = kepler_dram.groupby("label").last()

# remove the column time_normalized_rounded as it is not needed anymore
kepler_ram_joules_processed = kepler_ram_joules_processed.drop(columns=["time_normalized_rounded"])

# remove _system-under-evaluation from the column names
kepler_ram_joules_processed.columns = kepler_ram_joules_processed.columns.str.replace("_system-under-evaluation", "", regex=False)

# sort the index for readability in this order: Baseline, 5%, 10%, 50%, Baseline Persistence, 5% Persistence, 10% Persistence, 50% Persistence
kepler_ram_joules_processed = kepler_ram_joules_processed.reindex([
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
    "Base Persistence",
    "Scrape 5s Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "Persistence Istio",
])  


# drop indexes the experiments that are not needed e.g. Baseline, 0%, 5%, 10%, 50%, 1s, 30s
kepler_ram_joules_processed = kepler_ram_joules_processed.drop(index=[
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
])

# rename the columns to be more readable
kepler_ram_joules_processed = kepler_ram_joules_processed.rename(index={
    "Base Persistence": "Baseline",
    "Scrape 5s Persistence": "Scrape 5s",
    "5% Persistence": "5%",
    "10% Persistence": "10%",
    "50% Persistence": "50%",
    "Persistence Istio": "Istio",
})

kepler_ram_joules_processed_total = kepler_ram_joules_processed.copy()
kepler_ram_joules_processed_total["Total"] = kepler_ram_joules_processed.sum(axis=1)

# print as table
print("DRAM energy consumption in Joules per Experiment (label):")
print(kepler_ram_joules_processed_total.to_markdown())

# convert Joules to kWh
kepler_ram_joules_processed = kepler_ram_joules_processed / 3600000  # 1 kWh = 3.6 million Joules
print("Total energy consumption in kWh per Experiment (label) and percentage difference from label 'Baseline':")

for label in kepler_ram_joules_processed.index:
    if label == "Baseline":
        print(f"{label}: {kepler_ram_joules_processed.loc[label].sum()} kWh (100%)")
    else:
        percentage_difference = (kepler_ram_joules_processed.loc[label].sum() / kepler_ram_joules_processed.loc["Baseline"].sum()) * 100
        print(f"{label}: {kepler_ram_joules_processed.loc[label].sum()} kWh ({percentage_difference:.2f}%)")


DRAM energy consumption in Joules per Experiment (label):
| label     |   accounting |       ad |     cart |     checkout |   configfile |   currency |   elasticsearch |   email |     exporter |        flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |   istio-init |   istio-proxy |   jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |    kafka |       master |   opensearch |   opentelemetry-collector |      payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   worker |    Total |
|:----------|-------------:|---------:|---------:|-------------:|-------------:|-----------:|----------------:|--------:|-------------:|-------------:|-----------:|------------------:|-----------:|-----------------:|----------:|-----------------:|--------------:|-------------:|--------------:|---------:|------------------

Categorize the energy consumption into "App", "Auxiliary" and "System"

In [82]:
# | label            |   accounting |      ad |    cart |   checkout |   configfile |   copy-default-plugins |   currency |   elasticsearch |   elasticsearch-checker |   email |   flagd |   flagd-ui |   fraud-detection |   frontend |   frontend-proxy |   grafana |   image-provider |   init-config |   jaeger |   jaeger-agent-sidecar |   jaeger-collector |   jaeger-query |   kafka |   master |   opensearch |   opentelemetry-collector |   payment |   product-catalog |   prometheus-server |   quote |   recommendation |   shipping |   sysctl |   valkey-cart |   wait-for-kafka |   wait-for-valkey-cart |   worker |

kepler_mapping = {
    "accounting": "Primary",
    "ad": "Primary",
    "cart": "Primary",
    "checkout": "Primary",
    "configfile": "Primary",
    "copy-default-plugins": "Primary",
    "currency": "Primary",
    "elasticsearch": "Auxiliary",
    "elasticsearch-checker": "Auxiliary",
    "email": "Primary",
    "exporter": "Auxiliary",
    "flagd": "Auxiliary",
    "flagd-ui": "Auxiliary",
    "fraud-detection": "Primary",
    "frontend": "Primary",
    "frontend-proxy": "Primary",
    "grafana": "Auxiliary",
    "image-provider": "Primary",
    "init-config": "Primary",
    "istio-init": "Auxiliary",
    "istio-proxy": "Auxiliary",
    "jaeger": "Auxiliary",
    "jaeger-agent-sidecar": "Auxiliary",
    "jaeger-collector": "Auxiliary",
    "jaeger-query": "Auxiliary",
    "kafka": "Primary",
    "master": "Primary",
    "opensearch": "Auxiliary",
    "opentelemetry-collector": "Auxiliary",
    "payment": "Primary",
    "product-catalog": "Primary",
    "prometheus-server": "Auxiliary",
    "quote": "Primary",
    "recommendation": "Primary",
    "shipping": "Primary",
    "sysctl": "Auxiliary",
    "valkey-cart": "Primary",
    "wait-for-kafka": "Primary",
    "wait-for-valkey-cart": "Primary",
    "worker": "Primary",
}

#| label            |   accounting |          ad |   elasticsearch |   email |   flagd |   fraud-detection |     frontend |   frontend-proxy |     grafana |   image-provider |       kafka |   opensearch |   payment |   prometheus |   recommendation |   shipping |   valkey-cart |

storage_data_mapping = {
    "accounting": "Primary",
    "ad": "Primary",
    "elasticsearch": "Auxiliary",
    "email": "Primary",
    "flagd": "Auxiliary",
    "fraud-detection": "Primary",
    "frontend": "Primary",
    "frontend-proxy": "Primary",
    "grafana": "Auxiliary",
    "image-provider": "Primary",
    "kafka": "Primary",
    "opensearch": "Auxiliary",
    "payment": "Primary",
    "prometheus": "Auxiliary",
    "recommendation": "Primary",
    "shipping": "Primary",
    "valkey-cart": "Primary",
    "quote": "Primary",
    "product-catalog": "Primary",
    "opentelemetry-collector": "Auxiliary",
    "checkout": "Primary",
    "cart": "Primary",
    "jaeger": "Auxiliary",
    "currency": "Primary",
}

# | label            |   accounting |          ad |        cart |    checkout |    currency |   elasticsearch |   email |       flagd |   fraud-detection |   frontend |   frontend-proxy |     grafana |   image-provider |      jaeger |       kafka |   opensearch |   opentelemetry-collector |     payment |   product-catalog |   prometheus |   quote |   recommendation |   shipping |   valkey-cart |

network_data_mapping = {
    "accounting": "Primary",
    "ad": "Primary",
    "cart": "Primary",
    "checkout": "Primary",
    "currency": "Primary",
    "elasticsearch": "Auxiliary",
    "email": "Primary",
    "flagd": "Auxiliary",
    "fraud-detection": "Primary",
    "frontend": "Primary",
    "frontend-proxy": "Primary",
    "grafana": "Auxiliary",
    "image-provider": "Primary",
    "jaeger": "Auxiliary",
    "jaeger-collector": "Auxiliary", 
    "jaeger-query": "Auxiliary", 
    "otel-collector": "Auxiliary", 
    "kafka": "Primary",
    "opensearch": "Auxiliary",
    "opentelemetry-collector": "Auxiliary",
    "payment": "Primary",
    "product-catalog": "Primary",
    "prometheus": "Auxiliary",
    "quote": "Primary",
    "recommendation": "Primary",
    "shipping": "Primary",
    "valkey-cart": "Primary",
}


import pandas as pd

def categorize_energy_consumption(data: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    """
    Categorize and sum energy consumption based on component categories.

    Parameters:
    - data (pd.DataFrame): A DataFrame where rows represent measurements and columns represent component labels.
    - mapping (dict): A mapping from component label (column name) to a category (e.g., "Primary", "Auxiliary").

    Returns:
    - pd.DataFrame: A DataFrame with the same row indices as `data` and columns being the summed energy per category,
                    including an 'Uncategorized' column if needed.
    """
    # Initialize a DataFrame to store summed results
    categorized = pd.DataFrame(index=data.index)

    # Identify columns that are in the mapping
    mapped_cols = set(mapping.keys())
    data_cols = set(data.columns)
    unmapped_cols = list(data_cols - mapped_cols)

    # Print warning for unmapped columns
    if unmapped_cols:
        print(f"[WARNING] Uncategorized columns: {unmapped_cols}")

    # Iterate through unique categories
    for category in set(mapping.values()):
        cols = [col for col in data.columns if mapping.get(col) == category]
        categorized[category] = data[cols].sum(axis=1) if cols else 0

    # Add uncategorized columns
    if unmapped_cols:
        categorized["Uncategorized"] = data[unmapped_cols].sum(axis=1)
    
    return categorized


# Categorize the energy consumption for each dataset
network_categorized = categorize_energy_consumption(network_data_processed, network_data_mapping)
storage_categorized = categorize_energy_consumption(storage_data_processed, storage_data_mapping)
kepler_categorized = categorize_energy_consumption(kepler_data_joules_processed, kepler_mapping)

# Print the categorized energy consumption
print("\nNetwork Energy Consumption Categorized:")
print(network_categorized.to_markdown())
print("\nStorage Energy Consumption Categorized:")
print(storage_categorized.to_markdown())
print("\nKepler Energy Consumption Categorized:")
print(kepler_categorized.to_markdown())

# add a suffix to each set of columns
net = network_categorized.add_suffix(" (Network)")
cpu = kepler_categorized.add_suffix(" (CPU+Mem)")
sto = storage_categorized.add_suffix(" (Storage)")

categorized_combined_suffix = pd.concat([net, cpu, sto], axis=1) * 1000
categorized_combined_suffix = categorized_combined_suffix.sort_index(axis=1)

# Compute total energy consumption per row
categorized_combined_suffix["Total"] = categorized_combined_suffix.sum(axis=1)

# Calculate percentage change from the "Baseline" row
baseline_total = categorized_combined_suffix.loc["Baseline", "Total"]
categorized_combined_suffix["Total Change (%)"] = (
    (categorized_combined_suffix["Total"] - baseline_total) / baseline_total * 100
).round(2)

print("\nCombined Categorized Energy Consumption by origin in Wh:")
print(categorized_combined_suffix.round(2).to_markdown())

# merge the categorized dataframes and sum on same column name
categorized_combined = pd.concat([network_categorized, storage_categorized, kepler_categorized], axis=1)
categorized_combined = categorized_combined.T.groupby(categorized_combined.columns).sum().T


# Print the combined categorized energy consumption
#print("\nCombined Categorized Energy Consumption:")
#print(categorized_combined.to_markdown())

# add total summing up the categories for each experiment (label)
categorized_combined["Total"] = categorized_combined.sum(axis=1)

#categorized_combined_transpose = categorized_combined.transpose()
#print(categorized_combined_transpose.to_markdown())

categorized_combined_transpose_wh = categorized_combined.T.copy() * 1000  # convert kWh to Wh

print("\nCombined Categorized Energy Consumption in Wh:")
print(categorized_combined_transpose_wh.round(2).to_markdown())

# Combine all three sources on service/component label level
combined_per_label = pd.concat([
    network_data_processed,
    storage_data_processed,
    kepler_data_joules_processed
], axis=1)

# Group by column name (i.e., per service/component), and sum if duplicate columns exist
combined_per_label = combined_per_label.groupby(combined_per_label.columns, axis=1).sum()

combined_per_label["Total"] = combined_per_label.sum(axis=1)

combined_per_label_wh = combined_per_label * 1000  # convert kWh to Wh

print("\nCombined Total Energy Consumption per Service (Wh):")
print(combined_per_label_wh.round(2).to_markdown())


combined_per_label_wh_T = combined_per_label_wh.T  # transposed: services as rows
combined_per_label_wh_T["Total"] = combined_per_label_wh_T.sum(axis=1)

print("\nTransposed: Total Energy per Service across all Experiments (Wh):")
print(combined_per_label_wh_T.round(2).to_markdown())






Network Energy Consumption Categorized:
| label     |    Primary |   Auxiliary |
|:----------|-----------:|------------:|
| Baseline  | 0.00431986 |  0.00626336 |
| Scrape 5s | 0.0042207  |  0.00673323 |
| 5%        | 0.00427099 |  0.00736114 |
| 10%       | 0.00427637 |  0.00842385 |
| 50%       | 0.004142   |  0.0213423  |
| Istio     | 0.00447458 |  0.00581466 |

Storage Energy Consumption Categorized:
| label     |     Primary |   Auxiliary |
|:----------|------------:|------------:|
| Baseline  | 8.9344e-06  |  0.00130774 |
| Scrape 5s | 8.09909e-06 |  0.00134474 |
| 5%        | 8.13083e-06 |  0.00179979 |
| 10%       | 8.62861e-06 |  0.00228123 |
| 50%       | 8.73002e-06 |  0.00619983 |
| Istio     | 8.52023e-06 |  0.00106394 |

Kepler Energy Consumption Categorized:
| label     |   Primary |   Auxiliary |
|:----------|----------:|------------:|
| Baseline  | 0.024034  |  0.00464715 |
| Scrape 5s | 0.0235588 |  0.00490086 |
| 5%        | 0.023928  |  0.00516295 |
| 10%       | 

/tmp/ipykernel_3170092/214425870.py:207: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  combined_per_label = combined_per_label.groupby(combined_per_label.columns, axis=1).sum()


# Latex Tables for the Paper

In [109]:
latex_categorized_combined_suffix = categorized_combined_suffix.round(2).copy()


# order columns 
latex_categorized_combined_suffix = latex_categorized_combined_suffix.reindex(
    columns=[
        "Primary (Network)",
        "Primary (CPU+Mem)",
        "Primary (Storage)",
        "Auxiliary (Network)",
        "Auxiliary (CPU+Mem)",
        "Auxiliary (Storage)",
        "Total",
        "Total Change (%)"
    ]
)

latex_categorized_combined_suffix.columns = pd.MultiIndex.from_tuples([
    ("Primary", "Network"),
    ("Primary", "CPU+Mem"),
    ("Primary", "Storage"),
    ("Auxiliary", "Network"),
    ("Auxiliary", "CPU+Mem"),
    ("Auxiliary", "Storage"),
    ("Total", "Absolut"),
    ("Total", "%")
])




latex_categorized_combined_suffix.index.name = ""
latex = latex_categorized_combined_suffix.to_latex(
    escape=True,              # keep % symbols
    multicolumn=True,
    multicolumn_format='c',
    column_format='l|rrr|rrr|rrr',  # first col left-aligned, rest right-aligned
    float_format="%.2f",
    caption=r"Energy consumption of the experiment runs in Wh divided into the "
            r"categories Primary and Auxiliary services.",
    label=r"tab:energy",
)
print(latex)

\begin{table}
\caption{Energy consumption of the experiment runs in Wh divided into the categories Primary and Auxiliary services.}
\label{tab:energy}
\begin{tabular}{l|rrr|rrr|rrr}
\toprule
 & \multicolumn{3}{c}{Primary} & \multicolumn{3}{c}{Auxiliary} & \multicolumn{2}{c}{Total} \\
 & Network & CPU+Mem & Storage & Network & CPU+Mem & Storage & Absolut & \% \\
 &  &  &  &  &  &  &  &  \\
\midrule
Baseline & 4.32 & 24.03 & 0.01 & 6.26 & 4.65 & 1.31 & 40.58 & 0.00 \\
Scrape 5s & 4.22 & 23.56 & 0.01 & 6.73 & 4.90 & 1.34 & 40.77 & 0.46 \\
5\% & 4.27 & 23.93 & 0.01 & 7.36 & 5.16 & 1.80 & 42.53 & 4.80 \\
10\% & 4.28 & 23.77 & 0.01 & 8.42 & 5.85 & 2.28 & 44.61 & 9.94 \\
50\% & 4.14 & 22.21 & 0.01 & 21.34 & 11.11 & 6.20 & 65.01 & 60.20 \\
Istio & 4.47 & 22.29 & 0.01 & 5.81 & 9.89 & 1.06 & 43.54 & 7.30 \\
\bottomrule
\end{tabular}
\end{table}



In [ ]:
df = combined_per_label_wh_T.round(2).copy()   # rows = services, cols = experiments
df.index.name = ""                               # keeps top-left LaTeX cell blank

redacted = 0
# loop and drop rows where total is 0.00
for index, row in df.iterrows():
    if row["Total"] == 0.00:
        df.drop(index, inplace=True)
        redacted += 1
        

# Map every experiment column to either "In-Memory", "Elasticsearch", or "" (no group)
group_map = {
    "Baseline":   "",
    "5%":         "Tracing",
    "10%":        "Tracing",
    "50%":        "Tracing",
    "Scrape 5s":  "Monitoring",      # rename if you prefer “0 %” etc.
    "Istio":      "Service Mesh",
}

# Ensure the DataFrame has the columns in the order we want:
ordered_cols = [
    "Baseline", "5%", "10%", "50%", "Scrape 5s", "Istio"
]

df = df[ordered_cols]

top   = [group_map[c] for c in df.columns]
lower = df.columns                    # or any tidy-up you like
df.columns = pd.MultiIndex.from_arrays([top, lower])

latex = df.to_latex(
    escape=True,
    multicolumn=True,
    multicolumn_format='c',
    column_format='l|r|rrr|r|r',
    float_format="%.2f",
    caption=f"Energy usage in Wh per Service. Combination of CPU, memory, network and storage energy consumption. {redacted} services where redacted that used less then 0.01Wh",
    label="tab:energy_per_service",
)
print(latex)


\begin{table}
\caption{Energy usage in Wh per Service. Combination of CPU, memory, network and storage energy consumption. 14 services where redacted that used less then 0.01Wh}
\label{tab:energy_per_service}
\begin{tabular}{l|r|rrr|r|r|r}
\toprule
 &  & \multicolumn{3}{c}{Tracing} & Metrics & Service Mesh \\
label & Baseline & 5\% & 10\% & 50\% & Scrape 5s & Istio \\
 &  &  &  &  &  &  \\
\midrule
accounting & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 \\
ad & 0.23 & 0.23 & 0.23 & 0.20 & 0.22 & 0.21 \\
cart & 0.78 & 0.75 & 0.75 & 0.65 & 0.74 & 0.66 \\
elasticsearch & 0.90 & 3.08 & 5.38 & 25.84 & 0.86 & 0.74 \\
exporter & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\
flagd & 0.17 & 0.17 & 0.17 & 0.16 & 0.17 & 0.33 \\
fraud-detection & 0.04 & 0.04 & 0.04 & 0.04 & 0.04 & 0.04 \\
frontend & 16.80 & 16.81 & 16.65 & 15.79 & 16.59 & 15.75 \\
frontend-proxy & 4.12 & 4.09 & 4.07 & 3.86 & 4.01 & 3.77 \\
grafana & 0.07 & 0.07 & 0.07 & 0.06 & 0.07 & 0.07 \\
istio-proxy & 0.00 & 0.00 & 0.00 & 0.00 & 0.0